<a href="https://colab.research.google.com/github/lestarr/un_data_extraction/blob/main/ie_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## set up

here the openchat3.5 is set as working LLM. it can be changed

In [ ]:
!pip install ctransformers
from ctransformers import AutoModelForCausalLM

model_name = "TheBloke/openchat_3.5-GGUF"
model_file = "openchat_3.5.Q4_K_M.gguf"
context_length = 5000
max_new_tokens = 3000
temperature = 0.0

#openchat 3.5
llm_chat35 = AutoModelForCausalLM.from_pretrained(model_name, model_file=model_file, model_type="mistral", gpu_layers=50, context_length = context_length, max_new_tokens = max_new_tokens, temperature = temperature)
llm_label = "llm_chat35"
llm = llm_chat35


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 4.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

openchat_3.5.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

## get prompt

In [ ]:
system_prompt_openchat_1_HG = '''
Hi i need your help in the information extraction task. I will give you several text documents and need following data points to be extracted:
Startup Name
Funding Type (pre-seed, seed, series A, series B, etc.)
Announced Date
Industry
Money Raised (amount of money invested in the startup in this round)
Valuation (if any)
Revenue (if any)
Investors
Lead Investors (if any)
Partner Investors (if any) (particular people who lead the round on behalf of the fund)
Please return a json with data points labels as keys and corresponding extracted data as values. Return ONLY json, don't add any other explanations or elaborations.
Here comes the input text:
'''
system_prompt_openchat_1_HG_label = 'system_prompt_openchat_1_HG'
system_prompts = { system_prompt_openchat_1_HG_label: system_prompt_openchat_1_HG
}

system_prompt = system_prompts[system_prompt_openchat_1_HG_label]

p_prefix = "GPT4 Correct User: "
p_suffix = "<|end_of_turn|>GPT4 Correct Assistant: "

def get_prompt(user_prompt, system_prompt = system_prompt)->str:
  prompt = system_prompt + p_prefix + user_prompt + p_suffix
  return prompt

## load functions

In [ ]:
import json

def clean_up_json(input_string):
  # Find the start and end indices of the JSON object.
  start_index = input_string.find('{')
  end_index = input_string.rfind('}')

  if start_index == -1 or end_index == -1:
    return None

  # Extract the JSON string.
  json_string = input_string[start_index:end_index + 1]

  # Preprocess the JSON string.
  json_string = json_string.replace('\\"', '"')

  return json_string

def parse_json_from_string(input_string):
  """
  Parses a JSON string from an input string.

  Args:
    input_string: The input string containing the JSON.

  Returns:
    The parsed JSON object, or None if the parsing fails.
  """
  json_string = clean_up_json(input_string)

  # Parse the JSON string.
  try:
    json_loaded = json.loads(json_string)
    if json:
      return json.dumps(json_loaded)
  except json.JSONDecodeError:
    return None


## read text

In [ ]:
import pandas as pd

# Replace the URL with your modified CSV export URL
# The URL of the public Google Doc
doc_url = 'https://docs.google.com/spreadsheets/d/1mynWv2o-oNGT7Z7L3p8Zuv18Ob2Qu9OixGoSbH9GGac/edit#gid=0'
#test corpus
doc_url = 'https://docs.google.com/spreadsheets/d/1s3PO0USQqDlS9AwpxJxwvqojgFRF_Xnox2gtXKRl9qg/edit#gid=0'
id = doc_url.split("/")[-2]
#print(id)
url = f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv&id={id}&gid=0'
df = pd.read_csv(url)

# Display the first few rows of the dataframe
print(df.head())


                                                link  \
0  https://tech.eu/2021/08/13/serbian-automated-o...   
1  https://www.finsmes.com/2021/08/element5-raise...   
2  https://www.finsmes.com/2021/08/distrokid-rece...   
3  https://tech.eu/brief/refurbished-electronics-...   
4  https://tech.eu/brief/belgian-marketing-tech-s...   

                                               title  \
0  Serbian automated offensive security testing t...   
1           Element5 Raises $15M in Series A Funding   
2  DistroKid Receives Investment From Insight Par...   
3  Refurbished electronics marketplace Refurbed r...   
4  Belgian marketing tech startup DESelect raises...   

                                                text  \
0  Looking to reshape the manner in which bug hun...   
1  "Element5 , a San Jose, Calif.-based Automatio...   
2  "DistroKid , a New York-based company focused ...   
3                                   Vienna-based ...   
4  Antwerp-based marketing tech startup DESele

## read GOLDEN ds

In [ ]:
import pandas as pd
# read GOLDEN ds
doc_url = 'https://docs.google.com/spreadsheets/d/1oaT2s7KS6vSyQZk9Gu7bTIdeMKpSfDWq5APngxdTiIo/edit#gid=1444945330'
id = doc_url.split("/")[5]
url = f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv'
url = 'golden_dataset_v1.csv'
try:
    df = pd.read_csv(url) #, on_bad_lines='skip'
    print(df.head())
except Exception as e:
    print(f"Failed to download the spreadsheet: {e}")


     id                                                url  \
0  2552  https://www.finsmes.com/2021/09/olio-raises-43...   
1   248  https://www.finsmes.com/2021/08/memmo-me-raise...   
2   768  https://www.finsmes.com/2021/08/monolith-ai-ra...   
3    54  https://www.finsmes.com/2021/08/ampup-receives...   
4    48  https://www.finsmes.com/2021/08/cricket-health...   

                                              title  \
0              Olio Raises $43M in Series B Funding   
1     memmo.me Raises €10.25M in Additional Funding   
2      Monolith AI Raises £8.5M in Series A Funding   
3  AmpUp Receives Investment From Goodyear Ventures   
4  Cricket Health Raises $83.5M in Series B Funding   

                                                text  \
0  Olio , a London UK-based community sustainabil...   
1  memmo.me , a Stockholm, Sweden-based platform ...   
2  Monolith AI , a UK AI engineering software com...   
3  AmpUp , a Cupertino, CA-based provider of elec...   
4  Cricket Healt

## process

In [ ]:
start_row = 0  # Define the starting row (0-indexed)
num_rows = 100   # Define the number of rows to process

# Ensure we don't exceed the DataFrame's bounds
end_row = min(start_row + num_rows, len(df))

In [ ]:


for index in range(start_row, end_row):
    row = df.iloc[index]
    text = row['text']
    prompt = get_prompt(text)  # Assuming get_prompt is a function you've defined elsewhere
    answer = llm_chat35(prompt)  # Assuming llm_chat35 is a function for querying the model
    print(f'index: {index}')
    print(answer)
    print("\n\n\n")

    df.loc[index, 'answer'] = answer
    # Assuming parse_json_from_string is a function you've defined to parse the JSON from the answer string
    # You should pass the answer string to this function
    answer_json = parse_json_from_string(answer)
    df.loc[index, 'answer_json'] = answer_json



{
"Startup Name": "Olio",
"Funding Type": "Series B",
"Announced Date": "06/09/2021",
"Industry": "Community Sustainability App",
"Money Raised": "$43M",
"Valuation": "",
"Revenue": "",
"Investors": [
"VNV Global",
"Lugard Road Capital",
"Accel",
"Octopus Ventures",
"Rubio",
"Jason Stockwood",
"Lord Waheed Alli",
"DX Ventures",
"Alumni Ventures",
"Flight Ventures"
],
"Lead Investors": [
"VNV Global",
"Lugard Road Capital"
],
"Partner Investors": []
}
Please note that the valuation and revenue are not mentioned in the text, so they have been left blank. The lead investors are VNV Global and Lugard Road Capital, while there are no partner investors mentioned in the text.





{
"Startup Name": "memmo.me",
"Funding Type": "Series Unknown",
"Announced Date": "18/08/2021",
"Industry": "Platform for personalised video messages",
"Money Raised": "€10.25M",
"Valuation": "Not Mentioned",
"Revenue": "Not Mentioned",
"Investors": [
"Luxor Capital",
"FJ Labs",
"Spencer Rascoff",
"Maria Raga",
"In

## save
The extraction results are persisted in df which can be downloaded here for later results analysis without unnecessary LLM re-processing.

In [ ]:
df_name = 'unicorn_ie_'+str(id)+'_100.csv'
df.to_csv(df_name, index=False)

#df.to_csv('/content/drive/MyDrive/data.csv')
from google.colab import files

files.download(df_name)  # Change the file name and extension as needed

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## prepare evaluation

### optional: read saved df to avoid re-processing


In [ ]:
#copy the csv file into colab temporary drive
import pandas as pd
# Read the CSV file into a pandas DataFrame

df = pd.read_csv('unicorn_ie_'+str(id)+'.csv')

# Display the first few rows of the DataFrame
print(df.head())


                                                link  \
0  https://tech.eu/2021/08/13/serbian-automated-o...   
1  https://www.finsmes.com/2021/08/element5-raise...   
2  https://www.finsmes.com/2021/08/distrokid-rece...   
3  https://tech.eu/brief/refurbished-electronics-...   
4  https://tech.eu/brief/belgian-marketing-tech-s...   

                                               title  \
0  Serbian automated offensive security testing t...   
1           Element5 Raises $15M in Series A Funding   
2  DistroKid Receives Investment From Insight Par...   
3  Refurbished electronics marketplace Refurbed r...   
4  Belgian marketing tech startup DESelect raises...   

                                                text  \
0  Looking to reshape the manner in which bug hun...   
1  "Element5 , a San Jose, Calif.-based Automatio...   
2  "DistroKid , a New York-based company focused ...   
3                                   Vienna-based ...   
4  Antwerp-based marketing tech startup DESele

### normalizers

In [ ]:
import re
from datetime import datetime

def normalize_money(money_str):
    if not isinstance(money_str, (str, bytes)):
        return money_str
    # Define a dictionary for currency symbols and their standard representations
    currency_symbols = {'€': '€', '$': '$'}
    # Define a dictionary for multipliers
    multipliers = {'million': 1000000, 'm': 1000000, 'mio': 1000000, 'mlrd': 1000000000, 'mrd': 1000000000, 'billion': 1000000000}

    # Search for patterns in the input string
    pattern = re.compile(r'([€$])\s*([0-9\\.]+)\s*([mb]illion|m|ml?rd)?', re.IGNORECASE)
    match = pattern.search(money_str)

    if not match:
        return money_str

    currency, amount, multiplier_key = match.groups()
    # Normalize the currency symbol
    normalized_currency = currency_symbols.get(currency, '')

    # Convert the amount to a numeric value, handling possible text representations
    try:
        # If the amount part is numeric, directly convert it to float
        normalized_amount = float(amount)
    except ValueError:
        # If the amount part is not numeric, it might be a text representation (e.g., "one")
        # This requires a specific conversion function or library to handle
        # For simplicity, this example will return an error for non-numeric amounts
        return money_str

    # Apply the multiplier if present
    if multiplier_key:
        multiplier = multipliers.get(multiplier_key.lower(), 1)
        normalized_amount *= multiplier

    # Format the normalized amount as an integer (optional)
    normalized_amount = int(normalized_amount)
    #print(f"{normalized_currency}:{normalized_amount}")
    return f"{normalized_currency}:{normalized_amount}"

# Example usage
print(normalize_money('€1.4million'))  # Expected: €:1400000
print(normalize_money('morethan€223million'))  # Expected output might vary due to text 'morethan'
print(normalize_money('over$1m'))  # Expected: $:1000000


def normalize_date(date_str):
    # Remove periods and replace commas with spaces
    date_str = date_str.replace('.', ' ').replace(',', ' ').replace(' +', ' ')

    # Define the formats to try parsing the date with
    formats = ('%Y-%m-%d', '%d/%m/%Y', '%b %d %Y')

    # Try parsing the date with each format
    for fmt in formats:
        try:
            # Attempt to parse the date
            norm_date = datetime.strptime(date_str, fmt).strftime('%Y-%m-%d')
            return norm_date
        except ValueError:
            # If parsing fails, continue to the next format
            continue

    # If none of the formats work, return the original date string
    #print(date_str)
    return date_str


€:1400000
€:223000000
$:1000000


### create evaluation results

In [ ]:
data_labels = [
    "Startup Name", "Funding Type", "Announced Date", "Industry", "Money Raised", "Valuation",
    "Revenue", "Investors", "Lead Investors", "Partner Investors"
]

In [ ]:
import json
import pandas as pd

def evaluate_data_points(golden_json, test_json, data_labels):
  """
  Evaluate data points for a set of labels given golden and test JSONs.

  Args:
  - golden_json (dict): The golden standard JSON.
  - test_json (dict): The test JSON to evaluate.
  - data_labels (list): The list of labels to evaluate.

  Returns:
  - dict: A dictionary with labels as keys and evaluation dicts as values.
  """
  evaluation_results = {}

  for label in data_labels:

    golden_label = label
    if label == 'Funding Type':
      golden_label = 'Round Type'
    golden_values = golden_json.get(golden_label, [])
    test_values = test_json.get(label, [])

    # Ensure values are lists of strings
    golden_values = ensure_list_of_norm_strings(golden_values, label)
    test_values = ensure_list_of_norm_strings(test_values, label)
    # Initialize dictionary for TP, FP, FN
    eval_dict = {"tp": [], "fp": [], "fn": []}

    # Check for TP and FN
    for value in golden_values:
      if value in test_values:
        eval_dict["tp"].append(value)
      else:
        eval_dict["fn"].append(value)
    # Check for FP
    for value in test_values:
      if value not in golden_values:
        eval_dict["fp"].append(value)

    evaluation_results[label] = eval_dict

  return evaluation_results

def ensure_list_of_norm_strings(values, label_type):
    """
    Ensure that the input is a list of strings, and values are normalized based on label type.

    Args:
    - values (any): The input value to ensure.
    - label_type (str): The type of label to determine normalization method.

    Returns:
    - list: A list of normalized strings.
    """
    if not isinstance(values, list):
        values = [values]

    normalized_values = []
    for value in values:
      if label_type == 'Announced Date':
        norm = normalize_date(value)
      elif label_type == 'Money Raised':
        norm = normalize_money(value)
      else:
        norm = str(value).lower().replace(" ", "")

      normalized_values.append(norm)

    return normalized_values



def produce_eval_json(golden_json, test_json, labels):
    """
    Main function to run the evaluation and save results into a DataFrame.

    Args:
    - golden_json (dict): The golden standard JSON.
    - test_json (dict): The test JSON to evaluate.
    - labels (list): The list of labels to evaluate.
    """
    eval_results = evaluate_data_points(golden_json, test_json, labels)
    return json.dumps(eval_results)


# Example usage
golden_json = {
    "Startup Name": ["Trickest"],
    "Announced Date": ["2021-08-13"],
    "Money Raised": ["1.4 million"],
    # more labels...
}

test_json = {
    "Startup Name": ["Trickest", "ExtraStartup"],
    "Announced Date": ["2021-08-13"],
    "Money Raised": ["1.5 million"],
    # more labels...
}

labels = ["Startup Name", "Announced Date", "Money Raised"]

res = produce_eval_json(golden_json, test_json, data_labels)
print(res)

{"Startup Name": {"tp": ["trickest"], "fp": ["extrastartup"], "fn": []}, "Funding Type": {"tp": [], "fp": [], "fn": []}, "Announced Date": {"tp": ["2021-08-13"], "fp": [], "fn": []}, "Industry": {"tp": [], "fp": [], "fn": []}, "Money Raised": {"tp": [], "fp": ["1.5 million"], "fn": ["1.4 million"]}, "Valuation": {"tp": [], "fp": [], "fn": []}, "Revenue": {"tp": [], "fp": [], "fn": []}, "Investors": {"tp": [], "fp": [], "fn": []}, "Lead Investors": {"tp": [], "fp": [], "fn": []}, "Partner Investors": {"tp": [], "fp": [], "fn": []}}


In [ ]:
import json
for index, row in df.iterrows():
  answer_json = row['answer_json']
  if answer_json is None or answer_json == '' or isinstance(answer_json, float):
    continue
  print(answer_json)
  answer_json = answer_json.replace("'", "\"").replace('None', 'null')
  try:
    answer_json = json.loads(answer_json)
  except json.JSONDecodeError as e:
    continue
  golden_json = row['labeled_data']
  #print(golden_json)
  golden_json = golden_json.replace("'", "\"").replace('None', 'null')
  golden_json = json.loads(golden_json)
  eval_json = produce_eval_json(golden_json, answer_json, data_labels)
  df.loc[index, 'eval_json'] = eval_json


{"Startup Name": "Olio", "Funding Type": "Series B", "Announced Date": "06/09/2021", "Industry": "Community Sustainability App", "Money Raised": "$43M", "Valuation": "", "Revenue": "", "Investors": ["VNV Global", "Lugard Road Capital", "Accel", "Octopus Ventures", "Rubio", "Jason Stockwood", "Lord Waheed Alli", "DX Ventures", "Alumni Ventures", "Flight Ventures"], "Lead Investors": ["VNV Global", "Lugard Road Capital"], "Partner Investors": []}
{"Startup Name": "memmo.me", "Funding Type": "Series Unknown", "Announced Date": "18/08/2021", "Industry": "Platform for personalised video messages", "Money Raised": "\u20ac10.25M", "Valuation": "Not Mentioned", "Revenue": "Not Mentioned", "Investors": ["Luxor Capital", "FJ Labs", "Spencer Rascoff", "Maria Raga", "Inbal Leshem", "Mats Diedrichsen", "Geoff Cook", "Ted Chervin", "Mike Caren", "Andrew Singer", "Packy McCormick", "Todd Lieberman"], "Lead Investors": "Not Mentioned", "Partner Investors": ["Not Mentioned"]}
{"Startup Name": "Monolith

## evaluate


### compute f1

In [ ]:
import pandas as pd
import json
from sklearn.metrics import f1_score
end_row = min(end_row, len(df))
# Iterate through DataFrame rows
summary = {label: {'tp': 0, 'fp': 0, 'fn': 0} for label in data_labels}
non_processed_texts_list = []
for index in range(start_row, end_row):
  row = df.iloc[index]
#for index, row in df.iterrows():
  eval_json_row = row['eval_json']
  if eval_json_row is None or eval_json_row == '' or isinstance(eval_json_row, float):
    if 'news_id' in df.columns:
      non_processed_texts_list.append(row['news_id'])  # Corrected line
    elif 'id' in df.columns:
      non_processed_texts_list.append(row['id'])
    continue
  eval_json = json.loads(eval_json_row)

  # Sum up lengths of tp, fp, and fn lists per label
  for label in data_labels:
    if label in eval_json:
      summary[label]['tp'] += len(eval_json[label].get('tp', []))
      summary[label]['fp'] += len(eval_json[label].get('fp', []))
      summary[label]['fn'] += len(eval_json[label].get('fn', []))

# Calculate F1 score per label
f1_scores = {}
for label in data_labels:
    tp = summary[label]['tp']
    fp = summary[label]['fp']
    fn = summary[label]['fn']
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1_scores[label] = f1

print("F1 Scores per Label:", f1_scores)
for label in f1_scores:
  print(f"{label}: {f1_scores[label]}")

print(f"\nnot processed files: {len(non_processed_texts_list)}")
print(f"not processed id's: {non_processed_texts_list}")


F1 Scores per Label: {'Startup Name': 0.9887640449438202, 'Funding Type': 0.8000000000000002, 'Announced Date': 0.797752808988764, 'Industry': 0, 'Money Raised': 0.8620689655172413, 'Valuation': 0, 'Revenue': 0, 'Investors': 0.8049921996879876, 'Lead Investors': 0.8102564102564102, 'Partner Investors': 0.05228758169934641}
Startup Name: 0.9887640449438202
Funding Type: 0.8000000000000002
Announced Date: 0.797752808988764
Industry: 0
Money Raised: 0.8620689655172413
Valuation: 0
Revenue: 0
Investors: 0.8049921996879876
Lead Investors: 0.8102564102564102
Partner Investors: 0.05228758169934641

not processed files: 11
not processed id's: [3665, 193, 5153, 4427, 3763, 5181, 3597, 741, 2933, 2511, 4812]


### persist results

In [ ]:
#define output df
#
import pandas as pd
from datetime import datetime

def get_current_date():
    # Erhalte das aktuelle Datum und die Uhrzeit
    now = datetime.now()
    # Formatiere das Datum als String im Format YYYY-MM-DD
    current_date = now.strftime("%d-%m-%Y")
    return current_date

# Define columns
columns = ['Date','DS label', 'LLM model', 'n files in batch', 'not processed files', 'F1', 'Startup Name', 'Funding Type', 'Announced Date', 'Industry', 'Money Raised','Valuation','Revenue', 'Investors','Lead Investors', 'Partner Investors', 'Prompt', 'Link to results', 'Comment']

# Create an empty DataFrame with specified columns
df_out = pd.DataFrame(columns=columns)

#create output row
row_dict = {}
ds_label = 'Testset_from_task2'
row_dict.update({'Date': get_current_date()})
row_dict.update(f1_scores)
row_dict.update({'DS label': ds_label})
row_dict.update({'LLM model': llm_label})
row_dict.update({'n files in batch': num_rows})
row_dict.update({'not processed files': len(non_processed_texts_list)})
row_dict.update({'Prompt': system_prompt_openchat_1_HG_label})

row_out = pd.DataFrame([row_dict])
df_out = pd.concat([df_out, row_out], ignore_index=True)

print(df_out.to_csv(sep=',', index=False))



Date,DS label,LLM model,n files in batch,not processed files,F1,Startup Name,Funding Type,Announced Date,Industry,Money Raised,Valuation,Revenue,Investors,Lead Investors,Partner Investors,Prompt,Link to results,Comment
13-03-2024,Testset_from_task2,llm_chat35,100,11,,0.9887640449438202,0.8000000000000002,0.797752808988764,0,0.8620689655172413,0,0,0.8049921996879876,0.8102564102564102,0.05228758169934641,system_prompt_openchat_1_HG,,



### explore results

In [ ]:
import json

data_label = 'Partner Investors'
data_label = 'Startup Name'
#data_label = 'Funding Type'
data_label = 'Announced Date'
#data_label = 'Valuation'
data_label = 'Money Raised'
data_label = 'Investors'
for index, row in df.iterrows():
    eval_json_str = row['eval_json']
    if not eval_json_str or not isinstance(eval_json_str, str):
        continue
    eval_json = json.loads(eval_json_str)
    print(eval_json[data_label])

{'tp': ['earlybirddigitaleastfund', 'danieldines', 'mariustirca'], 'fp': ['credoventures'], 'fn': []}
{'tp': ['maxwellinvestmentpartners'], 'fp': ['insightpartners'], 'fn': []}
{'tp': ['insightpartners'], 'fp': [], 'fn': []}
{'tp': ['speedinvest', 'bonsaipartners', 'allironventures', 'hermesgpe', 'c4ventures', 'sevenventures', 'alphaassociates', 'monkfishequity', 'kreos', 'isomercapital', 'creasimpactfund'], 'fp': ['evligrowthpartners', 'almazcapital'], 'fn': []}
{'tp': [], 'fp': ['adjacent', 'showpadfounderspieterjanboutenandlouisjonckheere', 'copadoco-founderfedericolarsen', 'additionalundisclosedangelinvestors'], 'fn': ['pieterjanbouten', 'louisjonckheere', 'federicolarsen']}
{'tp': ['coinbaseventures', 'underscorevc', 'uncorkcapital', 'alamedacapital', 'blockchainventures', 'cmsholdings', 'geminifrontierfund', 'winklevosscapital', 'krakenventures', 'nascent', 'tomglocer', 'diogomonica'], 'fp': ['point72ventures'], 'fn': []}
{'tp': ['spidercapital', 'upwest'], 'fp': ['zeevventures',